#### A/B Test Forecast

This forecast is built for SaaS products with free trial periods. It will predict whether or not there will be a statistically significant difference in conversion rate between different groups of an A/B test.

The forecast first gathers historical data from the A/B test and analyzes how the conversion rate is trending for that historical data. Once the predicted conversion rate for the historical data is complete, the assumed conversion rates are applied to each group in the A/B test for future data.

Users have the ability to select different inputs for future test data, including...

1. Days the A/B test runs in the future
2. How many entries are added to the A/B test each day
3. Adjustments in conversion rate performance (+/- X%)

The forecast also includes the following assumptions...

1. There can be up to 4 different groups in the A/B test
2. Forecast allows up to 6 weeks of historical data

In [2]:
# Import packages
import pandas as pd
import numpy as np


In [4]:
# Upload historical data from test
raw = pd.read_csv('/Users/Matt/Desktop/Programming/Python/AB_TestForecast/ctct_data.csv', na_values=' ')


In [7]:
# Preview data
raw.info()
raw.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25483 entries, 0 to 25482
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   UID               25483 non-null  int64 
 1   GROUP_NAME        25483 non-null  object
 2   TRIAL_START_DATE  25483 non-null  object
 3   TRIAL_TENURE      25483 non-null  int64 
 4   WEEK_NUMBER       25483 non-null  object
 5   CONVERT_7         25483 non-null  int64 
 6   CONVERT_14        25483 non-null  int64 
 7   CONVERT_21        25483 non-null  int64 
 8   CONVERT_28        25483 non-null  int64 
 9   CONVERT_35        25483 non-null  int64 
 10  CONVERT_42        25483 non-null  int64 
 11  CONVERT_49        25483 non-null  int64 
 12  CONVERT_56        25483 non-null  int64 
dtypes: int64(10), object(3)
memory usage: 2.5+ MB


,UID,GROUP_NAME,TRIAL_START_DATE,TRIAL_TENURE,WEEK_NUMBER,CONVERT_7,CONVERT_14,CONVERT_21,CONVERT_28,CONVERT_35,CONVERT_42,CONVERT_49,CONVERT_56
0,1114728035448,4947a_Control,9/2/13,14,WEEK 7,0,0,0,0,0,0,0,0
1,1114567510056,4947b_Limit10,8/18/13,29,WEEK 4,0,0,0,0,0,0,0,0
2,1114497359375,4947a_Control,8/14/13,33,WEEK 4,0,0,0,0,0,0,0,0
3,1114329428592,4947a_Control,7/29/13,49,WEEK 2,1,1,1,1,1,1,1,1
4,1114539386764,4947a_Control,8/16/13,31,WEEK 4,0,0,0,0,0,0,0,0
